<a href="https://colab.research.google.com/github/wei-enwang/space-ham/blob/main/main_driver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import nltk
from nltk.corpus import words
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils import data
from preprocess import WholeData, BalancedData

import models
import utils

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/weinwang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
nltk.download('words')

device = "cuda" if torch.cuda.is_available() else "cpu"
assert device == "cuda"   # use gpu whenever you can!

seed = 32
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

[nltk_data] Downloading package words to /home/weinwang/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [13]:
plot_yes = True

# use one dataset for now
train_data_dir = "./data/enron1/"
test_data_dir = "./data/enron2/"
output_dir = "./output/"

# hyperparameters
batch_size = 64
hidden_size = 128
num_layer = 3
dropout = 0.5
learning_rate = 2e-5
epochs = 100
max_len = 300

In [8]:
vocab = set([str.lower() for str in words.words()])

# train_dataset = WholeData(train_data_dir, src_vocab=vocab, use_max_len=True, max_len=max_len)
# test_dataset = WholeData(test_data_dir, src_vocab=vocab, use_max_len=True, max_len=max_len)
# Balanced dataset
train_dataset = BalancedData(train_data_dir, src_vocab=vocab, use_max_len=True, max_len=max_len)
test_dataset = BalancedData(test_data_dir, src_vocab=vocab, use_max_len=True, max_len=max_len)

w2idx = train_dataset.src_v2id

embed = utils.load_pretrained_vectors(w2idx, "fastText/crawl-300d-2M.vec")
embed = torch.tensor(embed)

Number of ham emails: 1500, spam emails: 1500
Number of ham emails: 1496, spam emails: 1496
Loading pretrained vectors...
234378


0it [00:00, ?it/s]

There are 75835 / 234379 pretrained vectors found.


In [14]:
train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                                   num_workers=8, pin_memory=True)
test_dataloader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, 
                                  num_workers=8, pin_memory=True, drop_last=True)


In [15]:
model = models.spam_lstm(hidden_size=hidden_size, pretrained_embedding=embed, dropout=dropout).to(device)
loss_fn = nn.BCEWithLogitsLoss().to(device)
opt = Adam(model.parameters(), lr=learning_rate)

Using pretrained vectors...


In [ ]:
utils.train_test_scheme(train_dataloader, test_dataloader, model, loss_fn, opt, 
                        task_name="balancew2v+lstm300hid128", epochs=epochs, 
                        vis=plot_yes, print_every=1, img_dir=output_dir)



  1%|          | 1/100 [00:07<12:05,  7.32s/it]

Epoch 0
-------------------------------
Training loss: 0.693263, avg accuracy: 0.500047
Testing loss: 0.693185, avg accuracy: 0.502378


  2%|▏         | 2/100 [00:14<11:56,  7.31s/it]

Epoch 1
-------------------------------
Training loss: 0.693228, avg accuracy: 0.500000
Testing loss: 0.693273, avg accuracy: 0.498641


  3%|▎         | 3/100 [00:21<11:50,  7.32s/it]

...

Epoch 49
-------------------------------
Training loss: 0.557088, avg accuracy: 0.551909
Testing loss: 0.647215, avg accuracy: 0.521739


 51%|█████     | 51/100 [06:13<05:59,  7.33s/it]

Epoch 50
-------------------------------
Training loss: 0.556727, avg accuracy: 0.565919
Testing loss: 0.643585, avg accuracy: 0.520720


 52%|█████▏    | 52/100 [06:21<05:52,  7.34s/it]

Epoch 51
-------------------------------
Training loss: 0.557269, avg accuracy: 0.554664
Testing loss: 0.644417, avg accuracy: 0.521739


In [12]:
torch.save(model.state_dict(), output_dir+"balancew2v_lstm300hid128.pt")